# MGP and MI exchange volumes
The dataset *GME_MGP_MI_QUANTITA* refers to the total volume traded (both sell and buy) for each hours in the MGP and MI markets.

In [14]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])

volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,FLOWDATE,FLOWHOUR,MARKET,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
76950,20190101,1,MI1,1207.748,858.713,83.695,68.004,827.752,8.921,44.549,...,0.000,0.0,0.00,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.263
76962,20190101,1,MGP,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,...,1843.786,0.0,2438.00,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
77008,20190101,1,MI2,244.328,403.747,8.931,49.858,72.777,4.295,32.767,...,4.661,0.0,0.00,0.0,0.0,NaN,NaN,412.328,412.328,2018-12-31 16:41:34.837
76951,20190101,2,MI1,1262.958,913.156,77.863,104.702,896.259,5.944,27.843,...,0.356,0.0,0.00,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.267
76963,20190101,2,MGP,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,...,1845.050,0.0,2342.00,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,20241028,20,MGP,38477.375,39584.236,3217.961,6955.304,21084.356,988.297,2216.836,...,NaN,0.0,3515.00,0.0,0.0,0.0,3079.0,36652.115,39137.375,2024-10-27 13:10:39.487
9,20241028,21,MGP,36498.484,36861.302,2924.269,6456.992,19973.157,961.607,2131.188,...,NaN,0.0,3599.00,0.0,0.0,0.0,3286.0,34011.381,37158.484,2024-10-27 13:10:39.487
10,20241028,22,MGP,32895.419,33774.419,2663.385,5942.527,18457.664,895.019,1970.696,...,NaN,0.0,3609.13,0.0,0.0,0.0,3598.0,29592.419,33555.419,2024-10-27 13:10:39.487
11,20241028,23,MGP,29816.372,30225.877,2410.287,5414.058,16801.417,830.197,1781.350,...,NaN,0.0,3686.00,0.0,0.0,0.0,3598.0,26067.638,30030.672,2024-10-27 13:10:39.487


In [15]:
market_labels = volumes['MARKET'].unique()

print(market_labels)

['MI1' 'MGP' 'MI2' 'MI3' 'MI4' 'MI5' 'MI6' 'MI7']


In [16]:
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

In [17]:
mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.00,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.00,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.00,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.00,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.00,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28 19:00:00,38477.375,39584.236,3217.961,6955.304,21084.356,988.297,2216.836,2310.722,0.0,None,...,NaN,0.0,3515.00,0.0,0.0,0.0,3079.0,36652.115,39137.375,2024-10-27 13:10:39.487
2024-10-28 20:00:00,36498.484,36861.302,2924.269,6456.992,19973.157,961.607,2131.188,2157.977,0.0,None,...,NaN,0.0,3599.00,0.0,0.0,0.0,3286.0,34011.381,37158.484,2024-10-27 13:10:39.487
2024-10-28 21:00:00,32895.419,33774.419,2663.385,5942.527,18457.664,895.019,1970.696,2009.317,0.0,None,...,NaN,0.0,3609.13,0.0,0.0,0.0,3598.0,29592.419,33555.419,2024-10-27 13:10:39.487


In [27]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mgp_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-10-27 19:00:00,16082.403,13142.527,2024-10-26 13:10:38.657
2024-10-27 20:00:00,16660.286,13239.340,2024-10-26 13:10:38.657
2024-10-27 21:00:00,15867.098,12426.250,2024-10-26 13:10:38.657
2024-10-27 22:00:00,14925.215,11530.555,2024-10-26 13:10:38.657
2024-10-27 23:00:00,14014.838,10866.107,2024-10-26 13:10:38.657
2024-10-28 00:00:00,12948.854,10132.912,2024-10-26 13:10:38.657
2024-10-28 00:00:00,12137.957,9483.640,2024-10-27 13:10:39.477
2024-10-28 01:00:00,11607.237,9190.082,2024-10-27 13:10:39.477
2024-10-28 02:00:00,11435.051,8914.812,2024-10-27 13:10:39.480


In [28]:
mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])

In [30]:
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi1_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-10-26 19:00:00,928.469,1003.888,2024-10-26 18:22:36.960
2024-10-26 20:00:00,797.322,799.672,2024-10-26 18:22:36.960
2024-10-26 21:00:00,1024.813,1066.428,2024-10-26 18:22:36.960
2024-10-26 22:00:00,897.144,1271.264,2024-10-26 18:22:36.960
2024-10-26 23:00:00,497.350,1053.953,2024-10-26 18:22:36.960
2024-10-27 00:00:00,315.464,601.310,2024-10-27 18:22:35.730
2024-10-27 01:00:00,202.503,442.416,2024-10-27 18:22:35.733
2024-10-27 02:00:00,398.134,515.735,2024-10-27 18:22:35.733
2024-10-27 03:00:00,306.518,390.278,2024-10-27 18:22:35.737


In [21]:
mi2_volumes = volumes[volumes['MARKET'] == 'MI2']
mi2_volumes = mi2_volumes.drop(columns=['MARKET'])

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,244.328,403.747,8.931,49.858,72.777,4.295,32.767,33.184,0.0,None,...,4.661,0.0,0.000,0.0,0.0,NaN,NaN,412.328,412.328,2018-12-31 16:41:34.837
2019-01-01 01:00:00,344.540,376.232,6.435,46.820,200.790,1.779,29.451,59.265,0.0,None,...,5.133,0.0,0.000,0.0,0.0,NaN,NaN,387.540,387.540,2018-12-31 16:41:34.840
2019-01-01 02:00:00,409.502,398.710,6.260,48.704,256.279,1.546,22.431,63.184,0.0,None,...,3.530,0.0,0.000,0.0,0.0,NaN,NaN,409.502,409.502,2018-12-31 16:41:34.843
2019-01-01 03:00:00,410.986,437.378,5.970,39.583,226.192,1.622,18.614,51.005,0.0,None,...,1.908,0.0,0.000,0.0,0.0,NaN,NaN,448.986,448.986,2018-12-31 16:41:34.853
2019-01-01 04:00:00,359.230,347.735,7.611,27.183,213.312,1.429,13.187,33.508,0.0,None,...,2.486,0.0,0.000,0.0,0.0,NaN,NaN,359.230,359.230,2018-12-31 16:41:34.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27 20:00:00,596.218,616.734,48.904,88.007,213.075,18.217,65.003,131.053,0.0,None,...,NaN,0.0,36.000,278.0,0.0,0.0,0.0,NaN,NaN,2024-10-27 18:22:39.907
2024-10-27 21:00:00,631.386,565.860,90.639,112.702,221.836,15.709,19.812,63.531,0.0,None,...,NaN,0.0,0.000,141.1,0.0,0.0,0.0,NaN,NaN,2024-10-27 18:22:39.907
2024-10-27 22:00:00,421.344,417.975,69.584,97.005,106.160,18.384,14.024,83.930,0.0,None,...,NaN,0.0,35.000,15.9,0.0,0.0,0.0,NaN,NaN,2024-10-27 18:22:39.907


In [32]:
mi2_volumes_nord = mi2_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi2_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-10-26 19:00:00,159.622,363.522,2024-10-26 18:22:41.460
2024-10-26 20:00:00,182.682,376.682,2024-10-26 18:22:41.460
2024-10-26 21:00:00,104.748,146.586,2024-10-26 18:22:41.460
2024-10-26 22:00:00,160.925,132.749,2024-10-26 18:22:41.460
2024-10-26 23:00:00,185.694,117.768,2024-10-26 18:22:41.460
2024-10-27 00:00:00,592.811,422.018,2024-10-27 18:22:39.900
2024-10-27 01:00:00,531.009,440.318,2024-10-27 18:22:39.900
2024-10-27 02:00:00,535.840,136.580,2024-10-27 18:22:39.903
2024-10-27 03:00:00,216.504,158.905,2024-10-27 18:22:39.903


In [33]:
mi3_volumes = volumes[volumes['MARKET'] == 'MI3']
mi3_volumes = mi3_volumes.drop(columns=['MARKET'])
mi3_volumes_nord = mi3_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi3_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-10-25 18:00:00,231.755,238.355,2024-10-25 18:22:37.743
2024-10-25 19:00:00,244.239,244.339,2024-10-25 18:22:37.743
2024-10-25 20:00:00,206.445,186.345,2024-10-25 18:22:37.743
2024-10-25 21:00:00,186.373,173.721,2024-10-25 18:22:37.743
2024-10-25 22:00:00,133.492,149.741,2024-10-25 18:22:37.743
2024-10-25 23:00:00,147.329,177.301,2024-10-25 18:22:37.743
2024-10-26 12:00:00,279.748,127.019,2024-10-26 18:22:45.097
2024-10-26 13:00:00,264.698,210.756,2024-10-26 18:22:45.097
2024-10-26 14:00:00,272.634,243.931,2024-10-26 18:22:45.097


In [34]:
mi4_volumes = volumes[volumes['MARKET'] == 'MI4']
mi4_volumes = mi4_volumes.drop(columns=['MARKET'])
mi4_volumes_nord = mi4_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi4_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2021-09-19 10:00:00,35.446,335.960,2021-09-19 04:10:57.840
2021-09-19 11:00:00,27.397,48.069,2021-09-19 04:10:57.840
2021-09-19 12:00:00,13.714,41.316,2021-09-19 04:10:57.843
2021-09-19 13:00:00,20.175,12.943,2021-09-19 04:10:57.843
2021-09-19 14:00:00,27.808,58.166,2021-09-19 04:10:57.843
2021-09-19 15:00:00,17.959,10.590,2021-09-19 04:10:57.843
2021-09-19 16:00:00,42.771,94.523,2021-09-19 04:10:57.843
2021-09-19 17:00:00,110.805,244.656,2021-09-19 04:10:57.843
2021-09-19 18:00:00,120.737,350.911,2021-09-19 04:10:57.843
